<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


### Load the data

In [10]:
import pandas as pd

df = pd.read_csv('data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


### Clean the data if necessary (it will be)

In [11]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [12]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
df['Churn'].value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [8]:
df['Churn'].map({
    'No': 0,
    'Yes': 1
}).value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [14]:
df['gender'].value_counts()

Male      3555
Female    3488
Name: gender, dtype: int64

In [15]:
for column in df.columns:
    print('-'*5, column, '-'*5)
    print(df[column].value_counts())
    print()

----- customerID -----
5832-EXGTT    1
2607-DHDAK    1
9068-VPWQQ    1
9919-YLNNG    1
9408-HRXRK    1
             ..
4193-IBKSW    1
9026-RNUJS    1
8309-IEYJD    1
8148-BPLZQ    1
7943-RQCHR    1
Name: customerID, Length: 7043, dtype: int64

----- gender -----
Male      3555
Female    3488
Name: gender, dtype: int64

----- SeniorCitizen -----
0    5901
1    1142
Name: SeniorCitizen, dtype: int64

----- Partner -----
No     3641
Yes    3402
Name: Partner, dtype: int64

----- Dependents -----
No     4933
Yes    2110
Name: Dependents, dtype: int64

----- tenure -----
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: tenure, Length: 73, dtype: int64

----- PhoneService -----
Yes    6361
No      682
Name: PhoneService, dtype: int64

----- MultipleLines -----
No                  3390
Yes                 2971
No phone service     682
Name: MultipleLines, dtype: int64

----- InternetService -----
Fiber optic    3096
DSL      

In [26]:
for tcharge in df[df['TotalCharges'].str.contains('[^0-9.]')]['TotalCharges']:
    print(f'--{tcharge}--')

-- --
-- --
-- --
-- --
-- --
-- --
-- --
-- --
-- --
-- --
-- --


In [27]:
df['TotalCharges'].str.replace(' ', '0').astype(float)

0         29.85
1       1889.50
2        108.15
3       1840.75
4        151.65
         ...   
7038    1990.50
7039    7362.90
7040     346.45
7041     306.60
7042    6844.50
Name: TotalCharges, Length: 7043, dtype: float64

In [40]:
def wrangle(dataframe):
    df = dataframe.copy()

    # drop customerID - all samples unique
    df = df.drop(columns='customerID')

    # one-hot encode types of internet service
    #  not dropping the column because i am converting to yes/no (1/0) later
    df['fiber_optic'] = df['InternetService'] == 'Fiber optic'
    df['dsl'] = df['InternetService'] == 'DSL'
    
    # one-hot encode contract types
    df['contract_mtm'] = df['Contract'] == 'Month-to-month'
    df['contract_2yr'] = df['Contract'] == 'Two year'
    df['contract_1yr'] = df['Contract'] == 'One year'
    df = df.drop(columns='Contract')

    # one-hot encode payment method
    df['payment_echeck'] = df['PaymentMethod'] == 'Electronic check'
    df['payment_mcheck'] = df['PaymentMethod'] == 'Mailed check'
    df['payment_bank'] = df['PaymentMethod'] == 'Bank transfer (automatic)'
    df['payment_ccard'] = df['PaymentMethod'] == 'Credit card (automatic)'
    df = df.drop(columns='PaymentMethod')
    
    # encode binary categorical columns
    binary_columns = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                      'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                      'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling',
                      'Churn']
    for column in binary_columns:
        df[column] = df[column].map({
            'Male': 0,
            'Female': 1,
            'No': 0,
            'Yes': 1,
            'No phone service': 0,
            'Fiber optic': 1,
            'DSL': 1,
            'No internet service': 0,
        })

    # replace empty total with 0 then convert to float
    df['TotalCharges'] = df['TotalCharges'].str.replace(' ', '0').astype(float)
    
    for column in df.columns:
        df[column] = df[column].astype(float)
        
    return df

In [41]:
df_clean = wrangle(df)

In [79]:
df_clean

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Churn,fiber_optic,dsl,contract_mtm,contract_2yr,contract_1yr,payment_echeck,payment_mcheck,payment_bank,payment_ccard
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,34.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,45.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.0,0.0,1.0,1.0,24.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7039,1.0,0.0,1.0,1.0,72.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7040,1.0,0.0,1.0,1.0,11.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
7041,0.0,1.0,1.0,0.0,4.0,1.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [80]:
df_clean['Churn'].value_counts(normalize=True)

0.0    0.73463
1.0    0.26537
Name: Churn, dtype: float64

In [82]:
from tensorflow.keras.utils import normalize

target = 'Churn'
features = df_clean.drop(columns=target).columns

# X = normalize(df_clean[features].values)
X = df_clean[features].values
y = df_clean[target].values

print('X', X.shape, type(X))
print('y', y.shape, type(y))

X (7043, 26) <class 'numpy.ndarray'>
y (7043,) <class 'numpy.ndarray'>


### Create and fit a baseline Keras MLP model to the data.

In [83]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [86]:
# Define and compile the model
def create_model(additional_layers=0, nodes_per_layer=26, activation_per_layer='relu',
                 loss_function='binary_crossentropy', optimizer='adam'):
    model = Sequential()
    model.add(Dense(nodes_per_layer, activation='relu', input_dim=26))
    for _ in range(additional_layers):
        model.add(Dense(nodes_per_layer, activation=activation_per_layer))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [87]:
model = KerasClassifier(build_fn=create_model)

results = model.fit(X, y, epochs=10, validation_split=0.2)

Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 1s 138us/sample - loss: 18.1286 - accuracy: 0.6819 - val_loss: 0.8012 - val_accuracy: 0.6359
Epoch 2/10
5634/5634 [==============================] - 0s 59us/sample - loss: 0.9497 - accuracy: 0.7306 - val_loss: 0.8062 - val_accuracy: 0.7850
Epoch 3/10
5634/5634 [==============================] - 0s 58us/sample - loss: 0.6444 - accuracy: 0.7606 - val_loss: 0.8328 - val_accuracy: 0.7913
Epoch 4/10
5634/5634 [==============================] - 0s 58us/sample - loss: 0.6623 - accuracy: 0.7707 - val_loss: 0.4734 - val_accuracy: 0.7942
Epoch 5/10
5634/5634 [==============================] - 0s 60us/sample - loss: 0.5844 - accuracy: 0.7684 - val_loss: 0.8206 - val_accuracy: 0.7935
Epoch 6/10
5634/5634 [==============================] - 0s 58us/sample - loss: 0.5686 - accuracy: 0.7682 - val_loss: 0.4444 - val_accuracy: 0.8034
Epoch 7/10
5634/5634 [==============================] - 0s 59us/samp

### Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer

In [90]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping

stop = EarlyStopping(monitor='accuracy', min_delta=0.001, patience=5)

In [89]:
# tune number of nodes
params = {
    'nodes_per_layer': [26, 52, 78, 100]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
)

grid.fit(X, y, callbacks=[stop], epochs=10, validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Train on 4507 samples, validate on 1127 samples
Epoch 1/10
4507/4507 [==============================] - 1s 158us/sample - loss: 117.1196 - accuracy: 0.5372 - val_loss: 2.8665 - val_accuracy: 0.5714
Epoch 2/10
4507/4507 [==============================] - 0s 64us/sample - loss: 1.1539 - accuracy: 0.7087 - val_loss: 0.6724 - val_accuracy: 0.7604
Epoch 3/10
4507/4507 [==============================] - 0s 59us/sample - loss: 0.8914 - accuracy: 0.7322 - val_loss: 0.6357 - val_accuracy: 0.7711
Epoch 4/10
4507/4507 [==============================] - 0s 58us/sample - loss: 0.9396 - accuracy: 0.7382 - val_loss: 0.4965 - val_accuracy: 0.7844
Epoch 5/10
4507/4507 [==============================] - 0s 58us/sample - loss: 0.6424 - accuracy: 0.7535 - val_loss: 0.5246 - val_accuracy: 0.7915
Epoch 6/10
4507/4507 [==============================] - 0s 57us/sample - loss: 0.6585 - accuracy: 0.7575 - val_loss: 0.9997 - val_accuracy: 0.7799
Epoch 7/10
4507/4507 [==============================] - 0s 57us/sam

In [91]:
# tune number of layers
params = {
    'nodes_per_layer': [52],
    'additional_layers': [12, 8, 4, 0]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
)

grid.fit(X, y, callbacks=[stop], epochs=10, validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Train on 4507 samples, validate on 1127 samples
Epoch 1/10
4507/4507 [==============================] - 1s 301us/sample - loss: 0.5750 - accuracy: 0.7533 - val_loss: 0.5407 - val_accuracy: 0.7640
Epoch 2/10
4507/4507 [==============================] - 1s 114us/sample - loss: 0.5711 - accuracy: 0.7528 - val_loss: 0.5343 - val_accuracy: 0.7728
Epoch 3/10
4507/4507 [==============================] - 0s 106us/sample - loss: 0.5407 - accuracy: 0.7633 - val_loss: 0.5394 - val_accuracy: 0.7684
Epoch 4/10
4507/4507 [==============================] - 0s 100us/sample - loss: 0.5355 - accuracy: 0.7617 - val_loss: 0.5240 - val_accuracy: 0.7755
Epoch 5/10
4507/4507 [==============================] - 0s 96us/sample - loss: 0.5311 - accuracy: 0.7626 - val_loss: 0.5180 - val_accuracy: 0.7782
Epoch 6/10
4507/4507 [==============================] - 0s 95us/sample - loss: 0.5157 - accuracy: 0.7655 - val_loss: 0.5395 - val_accuracy: 0.7888
Epoch 7/10
4507/4507 [==============================] - 0s 108us/s

In [92]:
# tune batch size
params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10, 20, 50, 100],
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], epochs=10, validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Train on 4507 samples, validate on 1127 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4507/4507 [==============================] - 2s 350us/sample - loss: 1.5032 - accuracy: 0.7011 - val_loss: 0.6414 - val_accuracy: 0.6371
Epoch 2/10
4507/4507 [==============================] - 1s 228us/sample - loss: 0.6706 - accuracy: 0.7506 - val_loss: 0.5083 - val_accuracy: 0.7853
Epoch 3/10
4507/4507 [==============================] - 1s 227us/sample - loss: 0.5644 - accuracy: 0.7562 - val_loss: 0.6821 - val_accuracy: 0.7853
Epoch 4/10
4507/4507 [==============================] - 1s 214us/sample - loss: 0.5786 - accuracy: 0.7637 - val_loss: 0.9773 - val_accuracy: 0.7844
Epoch 5/10
4507/4507 [==============================] - 1s 212us/sample - loss: 0.5529 - accuracy: 0.7584 - val_loss: 0.4925 - val_accuracy: 0.7888
Epoch 6/10
4507/4507 [==============================] - 1s 213us/sample - loss: 0.4923 - accuracy: 0.7732 - val_loss: 0.4514 - val_accuracy: 0.7933
Epoch 7/10
4507/4507 [==============================] - 1s 211us/sample - loss: 0.4758 - accuracy: 0.7801 - val_loss: 0.444

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  1.9min finished


Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 2s 342us/sample - loss: 1.6593 - accuracy: 0.7153 - val_loss: 0.8584 - val_accuracy: 0.7750
Epoch 2/10
5634/5634 [==============================] - 1s 216us/sample - loss: 0.6200 - accuracy: 0.7591 - val_loss: 0.6298 - val_accuracy: 0.7693
Epoch 3/10
5634/5634 [==============================] - 1s 215us/sample - loss: 0.5533 - accuracy: 0.7613 - val_loss: 0.4666 - val_accuracy: 0.7906
Epoch 4/10
5634/5634 [==============================] - 1s 210us/sample - loss: 0.4993 - accuracy: 0.7776 - val_loss: 0.4675 - val_accuracy: 0.7921
Epoch 5/10
5634/5634 [==============================] - 1s 211us/sample - loss: 0.5030 - accuracy: 0.7822 - val_loss: 0.6792 - val_accuracy: 0.7764
Epoch 6/10
5634/5634 [==============================] - 1s 210us/sample - loss: 0.4917 - accuracy: 0.7824 - val_loss: 0.4424 - val_accuracy: 0.7899
Epoch 7/10
5634/5634 [==============================] - 1s 220us

In [93]:
# tune epochs
params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [10, 20, 50, 100],
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    scoring='accuracy',
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Train on 4507 samples, validate on 1127 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4507/4507 [==============================] - 2s 352us/sample - loss: 1.3949 - accuracy: 0.6936 - val_loss: 0.9318 - val_accuracy: 0.7693
Epoch 2/10
4507/4507 [==============================] - 1s 210us/sample - loss: 0.6691 - accuracy: 0.7440 - val_loss: 0.6893 - val_accuracy: 0.5634
Epoch 3/10
4507/4507 [==============================] - 1s 211us/sample - loss: 0.5589 - accuracy: 0.7575 - val_loss: 0.5956 - val_accuracy: 0.7817
Epoch 4/10
4507/4507 [==============================] - 1s 210us/sample - loss: 0.5245 - accuracy: 0.7657 - val_loss: 0.5198 - val_accuracy: 0.7817
Epoch 5/10
4507/4507 [==============================] - 1s 221us/sample - loss: 0.6015 - accuracy: 0.7570 - val_loss: 0.4974 - val_accuracy: 0.7844
Epoch 6/10
4507/4507 [==============================] - 1s 216us/sample - loss: 0.5129 - accuracy: 0.7655 - val_loss: 0.4861 - val_accuracy: 0.7631
Epoch 7/10
4507/4507 [==============================] - 1s 228us/sample - loss: 0.4895 - accuracy: 0.7783 - val_loss: 0.487

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  5.4min finished


Train on 5634 samples, validate on 1409 samples
Epoch 1/100
5634/5634 [==============================] - 2s 325us/sample - loss: 0.9322 - accuracy: 0.7201 - val_loss: 0.5174 - val_accuracy: 0.7764
Epoch 2/100
5634/5634 [==============================] - 1s 216us/sample - loss: 0.6126 - accuracy: 0.7543 - val_loss: 0.5322 - val_accuracy: 0.7899
Epoch 3/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.5198 - accuracy: 0.7707 - val_loss: 0.5910 - val_accuracy: 0.7821
Epoch 4/100
5634/5634 [==============================] - 1s 214us/sample - loss: 0.4951 - accuracy: 0.7719 - val_loss: 0.4484 - val_accuracy: 0.7793
Epoch 5/100
5634/5634 [==============================] - 1s 217us/sample - loss: 0.5069 - accuracy: 0.7728 - val_loss: 0.5821 - val_accuracy: 0.6317
Epoch 6/100
5634/5634 [==============================] - 1s 211us/sample - loss: 0.4706 - accuracy: 0.7822 - val_loss: 0.5142 - val_accuracy: 0.7793
Epoch 7/100
5634/5634 [==============================] - 1

In [94]:
# tune optimizer
params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'optimizer': ['adam', 'adadelta', 'nadam', 'rmsprop', 'ftrl']
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Train on 4507 samples, validate on 1127 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4507/4507 [==============================] - 2s 355us/sample - loss: 1.0693 - accuracy: 0.7116 - val_loss: 0.8296 - val_accuracy: 0.5004
Epoch 2/100
4507/4507 [==============================] - 1s 219us/sample - loss: 0.5694 - accuracy: 0.7621 - val_loss: 0.4708 - val_accuracy: 0.7959
Epoch 3/100
4507/4507 [==============================] - 1s 220us/sample - loss: 0.5348 - accuracy: 0.7610 - val_loss: 0.5085 - val_accuracy: 0.7853
Epoch 4/100
4507/4507 [==============================] - 1s 221us/sample - loss: 0.5010 - accuracy: 0.7768 - val_loss: 0.4499 - val_accuracy: 0.7986
Epoch 5/100
4507/4507 [==============================] - 1s 218us/sample - loss: 0.5086 - accuracy: 0.7699 - val_loss: 0.4500 - val_accuracy: 0.7933
Epoch 6/100
4507/4507 [==============================] - 1s 223us/sample - loss: 0.4915 - accuracy: 0.7766 - val_loss: 0.5957 - val_accuracy: 0.7853
Epoch 7/100
4507/4507 [==============================] - 1s 221us/sample - loss: 0.4974 - accuracy: 0.7759 - val_loss:

[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:  9.5min finished


5634/5634 [==============================] - 2s 324us/sample - loss: 1.1681 - accuracy: 0.7107 - val_loss: 0.5425 - val_accuracy: 0.7573
Epoch 2/100
5634/5634 [==============================] - 1s 215us/sample - loss: 0.5584 - accuracy: 0.7607 - val_loss: 0.5722 - val_accuracy: 0.7814
Epoch 3/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.5175 - accuracy: 0.7698 - val_loss: 0.6774 - val_accuracy: 0.7771
Epoch 4/100
5634/5634 [==============================] - 1s 211us/sample - loss: 0.4884 - accuracy: 0.7742 - val_loss: 0.4857 - val_accuracy: 0.7871
Epoch 5/100
5634/5634 [==============================] - 1s 210us/sample - loss: 0.4852 - accuracy: 0.7755 - val_loss: 0.5680 - val_accuracy: 0.7828
Epoch 6/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.4767 - accuracy: 0.7840 - val_loss: 0.4916 - val_accuracy: 0.7864
Epoch 7/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.4721 - accuracy: 0.7836 - val_loss:

In [99]:
from tensorflow.keras.optimizers import Adam

# Define and compile the model
def create_model(additional_layers=0, nodes_per_layer=26, activation_per_layer='relu',
                 loss_function='binary_crossentropy', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(nodes_per_layer, activation='relu', input_dim=26))
    for _ in range(additional_layers):
        model.add(Dense(nodes_per_layer, activation=activation_per_layer))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [100]:
# tune learning rate for Adam optimizer

params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001, 0.01, 0.1]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 4507 samples, validate on 1127 samples
Epoch 1/10
4507/4507 [==============================] - 2s 477us/sample - loss: 0.5727 - accuracy: 0.7455 - val_loss: 0.5278 - val_accuracy: 0.7791
Epoch 2/10
4507/4507 [==============================] - 1s 276us/sample - loss: 0.5303 - accuracy: 0.7715 - val_loss: 0.5242 - val_accuracy: 0.7720
Epoch 3/10
4507/4507 [==============================] - 1s 271us/sample - loss: 0.5081 - accuracy: 0.7772 - val_loss: 0.5412 - val_accuracy: 0.7356
Epoch 4/10
4507/4507 [==============================] - 1s 281us/sample - loss: 0.4967 - accuracy: 0.7741 - val_loss: 0.4669 - val_accuracy: 0.7862
Epoch 5/10
4507/4507 [==============================] - 1s 277us/sample - loss: 0.4830 - accuracy: 0.7726 - val_loss: 0.4573 - val_accuracy: 0.7933
Epoch 6/10
4507/4507 [==============================] - 1s 272us/sample - loss: 0.4750 - accuracy: 0.7783 - val_loss: 0.4618 - val_accuracy: 0.7977
Epoch 7/10
4507/4507 [==============================] - 1s 269us

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.9min finished


Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 3s 463us/sample - loss: 0.5880 - accuracy: 0.7510 - val_loss: 0.5367 - val_accuracy: 0.7693
Epoch 2/10
5634/5634 [==============================] - 2s 279us/sample - loss: 0.5394 - accuracy: 0.7636 - val_loss: 0.5423 - val_accuracy: 0.7729
Epoch 3/10
5634/5634 [==============================] - 2s 280us/sample - loss: 0.5267 - accuracy: 0.7680 - val_loss: 0.5091 - val_accuracy: 0.7800
Epoch 4/10
5634/5634 [==============================] - 2s 269us/sample - loss: 0.5273 - accuracy: 0.7678 - val_loss: 0.5194 - val_accuracy: 0.7764
Epoch 5/10
5634/5634 [==============================] - 1s 266us/sample - loss: 0.5173 - accuracy: 0.7707 - val_loss: 0.5024 - val_accuracy: 0.7828
Epoch 6/10
5634/5634 [==============================] - 2s 283us/sample - loss: 0.4933 - accuracy: 0.7764 - val_loss: 0.4668 - val_accuracy: 0.7771
Epoch 7/10
5634/5634 [==============================] - 2s 283us

In [101]:
# no momentum parameter tio tune for adam

In [102]:
from tensorflow.keras.optimizers import Adam

# Define and compile the model
def create_model(additional_layers=0, nodes_per_layer=26, activation_per_layer='relu',
                 loss_function='binary_crossentropy', learning_rate=0.001, network_weights_init='uniform'):
    model = Sequential()
    model.add(Dense(nodes_per_layer, activation='relu', input_dim=26, kernel_initializer=network_weights_init))
    for _ in range(additional_layers):
        model.add(Dense(nodes_per_layer, activation=activation_per_layer, kernel_initializer=network_weights_init))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=network_weights_init))

    optimizer = Adam(learning_rate=learning_rate)
    
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [103]:
# tune network weights initialization

params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['uniform', 'zero', 'glorot_normal']
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Train on 4507 samples, validate on 1127 samples
Epoch 1/10
4507/4507 [==============================] - 2s 491us/sample - loss: 0.5825 - accuracy: 0.7357 - val_loss: 0.5522 - val_accuracy: 0.7294
Epoch 2/10
4507/4507 [==============================] - 1s 295us/sample - loss: 0.5396 - accuracy: 0.7357 - val_loss: 0.5079 - val_accuracy: 0.7294
Epoch 3/10
4507/4507 [==============================] - 1s 281us/sample - loss: 0.5070 - accuracy: 0.7357 - val_loss: 0.4952 - val_accuracy: 0.7294
Epoch 4/10
4507/4507 [==============================] - 1s 280us/sample - loss: 0.4862 - accuracy: 0.7548 - val_loss: 0.4807 - val_accuracy: 0.7897
Epoch 5/10
4507/4507 [==============================] - 1s 300us/sample - loss: 0.4728 - accuracy: 0.7792 - val_loss: 0.4804 - val_accuracy: 0.7915
Epoch 6/10
4507/4507 [==============================] - 1s 286us/sample - loss: 0.4696 - accuracy: 0.7761 - val_loss: 0.4899 - val_accuracy: 0.7587
Epoch 7/10
4507/4507 [==============================] - 1s 270us

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.2min finished


Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 3s 447us/sample - loss: 0.5482 - accuracy: 0.7515 - val_loss: 0.5432 - val_accuracy: 0.7800
Epoch 2/10
5634/5634 [==============================] - 2s 276us/sample - loss: 0.5105 - accuracy: 0.7673 - val_loss: 0.4834 - val_accuracy: 0.7857
Epoch 3/10
5634/5634 [==============================] - 2s 284us/sample - loss: 0.4870 - accuracy: 0.7780 - val_loss: 0.4896 - val_accuracy: 0.7757
Epoch 4/10
5634/5634 [==============================] - 2s 289us/sample - loss: 0.4745 - accuracy: 0.7787 - val_loss: 0.4607 - val_accuracy: 0.7878
Epoch 5/10
5634/5634 [==============================] - 2s 310us/sample - loss: 0.4731 - accuracy: 0.7819 - val_loss: 0.4640 - val_accuracy: 0.7878
Epoch 6/10
5634/5634 [==============================] - 2s 303us/sample - loss: 0.4681 - accuracy: 0.7769 - val_loss: 0.5172 - val_accuracy: 0.7296
Epoch 7/10
5634/5634 [==============================] - 2s 291us

In [104]:
from tensorflow.keras.optimizers import Adam

# Define and compile the model
def create_model(additional_layers=0, nodes_per_layer=26, activation_per_layer='relu',
                 loss_function='binary_crossentropy', learning_rate=0.001, network_weights_init='uniform'):
    model = Sequential()
    model.add(Dense(nodes_per_layer, activation=activation_per_layer, input_dim=26, kernel_initializer=network_weights_init))
    for _ in range(additional_layers):
        model.add(Dense(nodes_per_layer, activation=activation_per_layer, kernel_initializer=network_weights_init))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=network_weights_init))

    optimizer = Adam(learning_rate=learning_rate)
    
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [105]:
# tune activation functions

params = {
    'nodes_per_layer': [52],
    'additional_layers': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['glorot_normal'],
    'activation_per_layer': ['relu', 'sigmoid', 'softmax'],
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train on 4507 samples, validate on 1127 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4507/4507 [==============================] - 2s 348us/sample - loss: 1.3580 - accuracy: 0.6982 - val_loss: 0.6246 - val_accuracy: 0.7799
Epoch 2/100
4507/4507 [==============================] - 1s 218us/sample - loss: 0.6489 - accuracy: 0.7504 - val_loss: 0.5862 - val_accuracy: 0.7835
Epoch 3/100
4507/4507 [==============================] - 1s 234us/sample - loss: 0.5472 - accuracy: 0.7679 - val_loss: 0.6448 - val_accuracy: 0.6060
Epoch 4/100
4507/4507 [==============================] - 1s 219us/sample - loss: 0.5653 - accuracy: 0.7597 - val_loss: 0.4702 - val_accuracy: 0.7879
Epoch 5/100
4507/4507 [==============================] - 1s 220us/sample - loss: 0.4865 - accuracy: 0.7803 - val_loss: 0.4956 - val_accuracy: 0.7773
Epoch 6/100
4507/4507 [==============================] - 1s 209us/sample - loss: 0.5075 - accuracy: 0.7739 - val_loss: 0.4686 - val_accuracy: 0.7933
Epoch 7/100
4507/4507 [==============================] - 1s 210us/sample - loss: 0.4872 - accuracy: 0.7817 - val_loss:

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  3.6min finished


5634/5634 [==============================] - 2s 323us/sample - loss: 1.0873 - accuracy: 0.7164 - val_loss: 0.7469 - val_accuracy: 0.7786
Epoch 2/100
5634/5634 [==============================] - 1s 214us/sample - loss: 0.5670 - accuracy: 0.7595 - val_loss: 0.5309 - val_accuracy: 0.7764
Epoch 3/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.5338 - accuracy: 0.7645 - val_loss: 0.4526 - val_accuracy: 0.7942
Epoch 4/100
5634/5634 [==============================] - 1s 212us/sample - loss: 0.4827 - accuracy: 0.7808 - val_loss: 0.4469 - val_accuracy: 0.7906
Epoch 5/100
5634/5634 [==============================] - 1s 214us/sample - loss: 0.4776 - accuracy: 0.7790 - val_loss: 0.4516 - val_accuracy: 0.7899
Epoch 6/100
5634/5634 [==============================] - 1s 218us/sample - loss: 0.5169 - accuracy: 0.7764 - val_loss: 0.4688 - val_accuracy: 0.7842
Epoch 7/100
5634/5634 [==============================] - 1s 213us/sample - loss: 0.4667 - accuracy: 0.7827 - val_loss:

In [108]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout

# Define and compile the model
def create_model(additional_layers=0, nodes_per_layer=26, activation_per_layer='relu',
                 loss_function='binary_crossentropy', learning_rate=0.001,
                 network_weights_init='uniform', dropout_rate=0.1):
    model = Sequential()
    model.add(Dense(nodes_per_layer, activation=activation_per_layer, input_dim=26, kernel_initializer=network_weights_init))
    for _ in range(additional_layers):
        model.add(Dropout(rate=dropout_rate))
        model.add(Dense(nodes_per_layer, activation=activation_per_layer, kernel_initializer=network_weights_init))
    model.add(Dense(1, activation='sigmoid', kernel_initializer=network_weights_init))

    optimizer = Adam(learning_rate=learning_rate)
    
    model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [109]:
# tune dropout

params = {
    'nodes_per_layer': [4],
    'batch_size': [10],
    'epochs': [100],
    'learning_rate': [0.001],
    'network_weights_init': ['glorot_normal'],
    'activation_per_layer': ['relu'],
    'dropout_rate': [0.1, 0.2, 0.3]
}

grid = GridSearchCV(
    estimator=KerasClassifier(build_fn=create_model),
    param_grid=params,
    n_jobs=1,
    cv=5,
    verbose=1
)

grid.fit(X, y, callbacks=[stop], validation_split=0.2)

print()
print('Best score:', grid.best_score_)
print('Best params:', grid.best_params_)
print()

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Train on 4507 samples, validate on 1127 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


4507/4507 [==============================] - 1s 267us/sample - loss: 54.4153 - accuracy: 0.6576 - val_loss: 1.6594 - val_accuracy: 0.5049
Epoch 2/100
4507/4507 [==============================] - 1s 185us/sample - loss: 0.8717 - accuracy: 0.6854 - val_loss: 1.3436 - val_accuracy: 0.7737
Epoch 3/100
4507/4507 [==============================] - 1s 176us/sample - loss: 0.6267 - accuracy: 0.7699 - val_loss: 1.5895 - val_accuracy: 0.5084
Epoch 4/100
4507/4507 [==============================] - 1s 171us/sample - loss: 0.7285 - accuracy: 0.7533 - val_loss: 1.9144 - val_accuracy: 0.7666
Epoch 5/100
4507/4507 [==============================] - 1s 178us/sample - loss: 0.7257 - accuracy: 0.7608 - val_loss: 0.8907 - val_accuracy: 0.7844
Epoch 6/100
4507/4507 [==============================] - 1s 172us/sample - loss: 0.7015 - accuracy: 0.7621 - val_loss: 0.5094 - val_accuracy: 0.7693
Epoch 7/100
4507/4507 [==============================] - 1s 169us/sample - loss: 0.6007 - accuracy: 0.7695 - val_loss

[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  2.9min finished


5634/5634 [==============================] - 2s 270us/sample - loss: 30.2095 - accuracy: 0.7265 - val_loss: 0.7516 - val_accuracy: 0.4010
Epoch 2/100
5634/5634 [==============================] - 1s 174us/sample - loss: 0.6998 - accuracy: 0.4796 - val_loss: 0.6572 - val_accuracy: 0.7296
Epoch 3/100
5634/5634 [==============================] - 1s 169us/sample - loss: 0.6260 - accuracy: 0.7359 - val_loss: 0.6074 - val_accuracy: 0.7296
Epoch 4/100
5634/5634 [==============================] - 1s 169us/sample - loss: 0.5935 - accuracy: 0.7359 - val_loss: 0.5905 - val_accuracy: 0.7296
Epoch 5/100
5634/5634 [==============================] - 1s 169us/sample - loss: 0.5823 - accuracy: 0.7359 - val_loss: 0.5852 - val_accuracy: 0.7296
Epoch 6/100
5634/5634 [==============================] - 1s 171us/sample - loss: 0.5787 - accuracy: 0.7359 - val_loss: 0.5839 - val_accuracy: 0.7296
Epoch 7/100
5634/5634 [==============================] - 1s 170us/sample - loss: 0.5777 - accuracy: 0.7359 - val_loss

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?